In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from copy import deepcopy
import numpy as np
import pandas as pd

# Load Iris dataset
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Use only 2 classes for simplicity (make it binary classification)
binary_mask = y != 2
X = X[binary_mask]
y = y[binary_mask]

# Split into train and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Initialize SGDClassifier (supports partial_fit)
model = SGDClassifier(loss="log_loss", max_iter=1, warm_start=True, random_state=42)

In [3]:
from copy import deepcopy
from sklearn.metrics import accuracy_score

# For tracking best model
best_model = None
best_val_acc = 0
patience = 3
no_improve_epochs = 0
val_acc_history = []

# First call to partial_fit requires classes explicitly
classes = np.unique(y)


In [4]:
best_model = None
best_val_acc = 0
patience = 3
no_improve_epochs = 0
val_acc_history = []
train_acc_history = []

# Needed for first call of partial_fit
classes = np.unique(y_train)

# Training loop
n_epochs = 50
for epoch in range(n_epochs):
    # Train for one epoch
    model.partial_fit(X_train_scaled, y_train, classes=classes)
    
    # Predict on train and validation sets
    y_train_pred = model.predict(X_train_scaled)
    y_val_pred = model.predict(X_val_scaled)

    # Calculate accuracies
    train_acc = accuracy_score(y_train, y_train_pred)
    val_acc = accuracy_score(y_val, y_val_pred)
    
    train_acc_history.append(train_acc)
    val_acc_history.append(val_acc)

    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.4f} | Val Acc = {val_acc:.4f}")
    
    # Check for improvement
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model = deepcopy(model)
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
    
    # Early stop
    if no_improve_epochs >= patience:
        print(f"⛔ Early stopping triggered at epoch {epoch+1}")
        break

Epoch 1: Train Acc = 1.0000 | Val Acc = 1.0000
Epoch 2: Train Acc = 1.0000 | Val Acc = 1.0000
Epoch 3: Train Acc = 1.0000 | Val Acc = 1.0000
Epoch 4: Train Acc = 1.0000 | Val Acc = 1.0000
⛔ Early stopping triggered at epoch 4
